# Lecture 16의 데모

### Lecture 16의 데모에 대한 코드

참고자료: 

https://github.com/data-8/materials-sp22/blob/main/lec/lec29.ipynb

https://github.com/data-8/materials-sp22/blob/main/lec/lec30.ipynb


In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/ITEC419-fa22/lec

Mounted at /content/gdrive
/content/gdrive/MyDrive/ITEC419-fa22/lec


In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [ ]:
# Some functions for plotting. You don't have to understand how any
# of the functions in this cell work, since they use things we 
# haven't learned about in Data 8.


def resize_window(lim=3.5):
    plots.xlim(-lim, lim)
    plots.ylim(-lim, lim)
    
def draw_line(slope=0, intercept=0, x=make_array(-4, 4), color='#1e90ff'):
    y = x*slope + intercept
    plots.plot(x, y, color=color, lw=3)
    
def draw_vertical_line(x_position, color='black'):
    x = make_array(x_position, x_position)
    y = make_array(-4, 4)
    plots.plot(x, y, color=color, lw=3)
    
def make_correlated_data(r):
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    return x, y

def r_scatter(r):
    """Generate a scatter plot with a correlation approximately r"""
    plots.figure(figsize=(5,5))
    x, y = make_correlated_data(r)
    plots.scatter(x, y, color='darkblue', s=20)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)
    
def r_table(r):
    """
    Generate a table of 1000 data points with a correlation approximately r
    """
    np.random.seed(8)
    x, y = make_correlated_data(r)
    return Table().with_columns('x', x, 'y', y)

## **Approach to Prediction** (Lecture 7)

In [ ]:
# Note: Child heights are the **adult** heights of children in a family
families = Table.read_table('family_heights.csv')
families

In [ ]:
# convert inches to cm


In [ ]:
parent_avgs = (families.column('father') + families.column('mother'))/2

In [ ]:
heights = Table().with_columns(
    'Parent Average', parent_avgs,
    'Child', families.column('child'),
)
heights

In [ ]:
heights.scatter('Parent Average', 'Child')

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([171.0, 171.0], [135.0, 205.0], color='red', lw=2)
plots.plot([173.0, 173.0], [135.0, 205.0], color='red', lw=2);

In [ ]:
nearby = heights.where('Parent Average', are.between(171.0, 173.0))
nearby_mean = np.average(nearby.column('Child'))
nearby_mean

In [ ]:
heights.scatter('Parent Average', 'Child')
plots.plot([171.0, 171.0], [135.0, 205.0], color='red', lw=2)
plots.plot([173.0, 173.0], [135.0, 205.0], color='red', lw=2)
plots.scatter(172.0, nearby_mean, color='red', s=50);

In [ ]:
def predict_child(h):
    """Predict the height of a child whose parents have a parent average height of p_avg.
    
    The prediction is the average height of the children whose parent average height is
    in the range p_avg plus or minus 1.0.
    """
    nearby = heights.where('Parent Average', are.between(h - 1.0, h + 1.0))
    return np.average(nearby.column('Child'))

In [ ]:
heights_with_predictions = heights.with_columns(
    'Prediction', heights.apply(predict_child, 'Parent Average'))

In [ ]:
heights_with_predictions.scatter('Parent Average')

## **Association**

In [ ]:
hybrid = Table.read_table('hybrid.csv')
hybrid

In [ ]:
hybrid.sort('msrp', descending=True)

In [ ]:
hybrid.scatter('mpg', 'msrp')

In [ ]:
hybrid.scatter('acceleration', 'msrp')

In [ ]:
suv = hybrid.where('class', 'SUV')
suv.num_rows

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
hybrid.with_column('SUV', hybrid.column('class') == 'SUV').scatter('mpg', 'msrp', group='SUV')

In [ ]:
def standard_units(x):
    "Convert any array of numbers to standard units."
    return (x - np.average(x)) / np.std(x)

In [ ]:
Table().with_columns(
    'mpg (standard units)',  standard_units(suv.column('mpg')), 
    'msrp (standard units)', standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
Table().with_columns(
    'acceleration (standard units)', standard_units(suv.column('acceleration')), 
    'msrp (standard units)',         standard_units(suv.column('msrp'))
).scatter(0, 1)
plots.xlim(-3, 3)
plots.ylim(-3, 3);

## **Correlation**

In [ ]:
r_scatter(-0.5)

In [ ]:
x = np.arange(1, 7, 1)
y = make_array(2, 3, 1, 5, 2, 7)
t = Table().with_columns(
        'x', x,
        'y', y
    )
t

In [ ]:
t.scatter('x', 'y', s=30, color='red')

In [ ]:
t = t.with_columns(
        'x (standard units)', standard_units(x),
        'y (standard units)', standard_units(y)
    )
t

In [ ]:
t.scatter(2, 3, s=30, color='red')

In [ ]:
t = t.with_columns(
    'product of standard units', t.column(2) * t.column(3))
t

In [ ]:
# r is the average of the products of the standard units

r = np.average(t.column(2) * t.column(3))
r

In [ ]:
def correlation(t, x, y):
    """t is a table; x and y are column labels"""
    x_in_standard_units = standard_units(t.column(x))
    y_in_standard_units = standard_units(t.column(y))
    return np.average(x_in_standard_units * y_in_standard_units)

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
suv.scatter('mpg', 'msrp')

In [ ]:
correlation(suv, 'mpg', 'msrp')

In [ ]:
suv.scatter('acceleration', 'msrp')

In [ ]:
correlation(suv, 'acceleration', 'msrp')

### **Switching Axes**

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
t.scatter('x', 'y', s=30, color='red')

In [ ]:
t.scatter('y', 'x', s=30, color='red')

In [ ]:
correlation(t, 'y', 'x')

### **Nonlinearity**

In [ ]:
new_x = np.arange(-4, 4.1, 0.5)
nonlinear = Table().with_columns(
        'x', new_x,
        'y', new_x**2
    )
nonlinear.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(nonlinear, 'x', 'y')

### **Outliers**

In [ ]:
line = Table().with_columns(
        'x', make_array(1, 2, 3, 4),
        'y', make_array(1, 2, 3, 4)
    )
line.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(line, 'x', 'y')

In [ ]:
outlier = Table().with_columns(
        'x', make_array(1, 2, 3, 4, 5),
        'y', make_array(1, 2, 3, 4, 0)
    )
outlier.scatter('x', 'y', s=30, color='r')

In [ ]:
correlation(outlier, 'x', 'y')

### **Ecological Correlations**

In [ ]:
sat2014 = Table.read_table('sat2014.csv').sort('State')
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math')

In [ ]:
correlation(sat2014, 'Critical Reading', 'Math')

In [ ]:
def rate_code(x):
    if x <= 25:
        return 'low'
    elif x <= 75:
        return 'medium'
    else:
        return 'high'

In [ ]:
rate_codes = sat2014.apply(rate_code, 'Participation Rate')

In [ ]:
sat2014 = sat2014.with_columns('Rate Code', rate_codes)
sat2014

In [ ]:
sat2014.scatter('Critical Reading', 'Math', group='Rate Code')

In [ ]:
sat2014.where('Rate Code', 'low').show()

## **Prediction Lines**

### **r = 0.99**

In [ ]:
example = r_table(0.99)
example.show(3)

In [ ]:
example.scatter('x', 'y')
resize_window()

In [ ]:
def nn_prediction_example(x_val):
    """ Predicts y-value for x based on the example table """
    neighbors = example.where('x', are.between(x_val - .25, x_val + .25))
    return np.mean(neighbors.column('y'))   

In [ ]:
nn_prediction_example(-2.25)

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1)
resize_window()

### **r = 0**

In [ ]:
example = r_table(0)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = example.with_columns(
    'Predicted y', 
    example.apply(nn_prediction_example, 'x'))

In [ ]:
example.scatter('x')
draw_line(slope = 0)
resize_window()

### **r = 0.5**

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()

In [ ]:
example = r_table(0.5)
example.scatter('x', 'y')
resize_window()
draw_vertical_line(1.5)
draw_line(slope=1, intercept=0, color='red')

In [ ]:
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=1, color='red')
draw_vertical_line(1.5)
resize_window()

In [ ]:
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.5, intercept=0)
resize_window()

###  **r = 0.7**

In [ ]:
example = r_table(0.7)
example = example.with_column('Predicted y', example.apply(nn_prediction_example, 'x'))
example.scatter('x')
draw_line(slope=1, intercept=0, color='red')
draw_line(slope=0.7, intercept=0, color='dodgerblue')
resize_window()